In [3]:
## Author: Mohsen Ahmadkhani
## Lab 1
## Arc I

#Fetch all nearby coffee shops (dist < 5000 m) from Goople Places

import requests
from urllib.parse import urlencode
import pandas as pd 
import geopandas as gpd
from geopandas.tools import sjoin
from shapely.geometry import Point
import matplotlib.pyplot as plt
import folium
 
api_key = 'API KEY HERE!' # Luke, please put your API token here to run the code successfully

lat, lon = 44.9779006, -93.1831530 #my home in Falcon Heights

url_base = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
params = {
    "key": api_key,
    "location": f"{lat},{lon}",
    "radius": 5000,
    "keyword": "coffee shop"
}

encoded_params = urlencode(params)
query_url =  url_base + encoded_params 

res = requests.get(query_url)
res_json = res.json()

df_l = []

for i in range(len(res_json['results'])):
    df_l.append({
    'name' : res_json['results'][i]['name'],
    'business_status': res_json['results'][i]['business_status'],
    'open_now' : res_json['results'][i]['opening_hours']['open_now'], # T/F
    'rating' : res_json['results'][i]['rating'],
    'address' : res_json['results'][i]['vicinity'],
    'lat' : res_json['results'][i]['geometry']['location']['lat'],
    'lng' : res_json['results'][i]['geometry']['location']['lng']
    })

coffee_shops = pd.DataFrame(df_l)

coffee_shops_gdf = gpd.GeoDataFrame(coffee_shops, geometry=gpd.points_from_xy(coffee_shops.lng, coffee_shops.lat))
coffee_shops_gdf.crs = 4326 #setting the Geodetic coordinate system this will be for all

# Plot the results

# base_map = coffee_shops_gdf.plot(figsize = (15,15))

# ax = gpd.GeoDataFrame([{'geom':home}], geometry = 'geom').plot(ax = base_map, color = 'red', marker = '+')
# ax.annotate("Mohsen's home", xy=(home.x, home.y), xytext=(3, 3), textcoords="offset points")

# for el in coffee_shops_gdf.iterrows():
#     ax.annotate(el[1][0], xy=(el[1][6], el[1][5]), xytext=(3, 3), textcoords="offset points")
# plt.show()

coffee_shops_gdf.head()


,name,business_status,open_now,rating,address,lat,lng,geometry
0,Alle Aamin Coffee Shop,OPERATIONAL,True,4.5,"609 Cedar Ave, Minneapolis",44.967374,-93.247063,POINT (-93.24706 44.96737)
1,Midway Coffee and Tea,OPERATIONAL,True,5.0,"1607 University Ave W, St Paul",44.956069,-93.168279,POINT (-93.16828 44.95607)
2,Lori's Coffee House,OPERATIONAL,False,4.6,"1441 Cleveland Ave N, St Paul",44.984514,-93.187575,POINT (-93.18757 44.98451)
3,Starbucks,OPERATIONAL,False,3.8,"1536 Hewitt Ave, St Paul",44.964455,-93.166626,POINT (-93.16663 44.96446)
4,Up Cafe and Up Coffee Roasters,OPERATIONAL,False,4.8,"1901 Traffic St NE, Minneapolis",44.992616,-93.225131,POINT (-93.22513 44.99262)


In [4]:
# plot an interactive map with folium
# The blue circle shows the vicinity of 3kms of my home
m = folium.Map(location = [44.9779006, -93.1831530], tiles='OpenStreetMap' , zoom_start = 12) # tiles="Stamen Toner"
home = Point(-93.1831530, 44.9779006) # build my home pont
mohsens_home = gpd.GeoDataFrame([{'geom':home}], geometry = 'geom')
mohsens_home.crs = 4326

for _, r in coffee_shops_gdf.iterrows():

    sim_geo = gpd.GeoSeries(r['geometry']) #.simplify(tolerance=0.001) 
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, 
                           style_function = lambda x: {'color': 'red', 'weight': 1,  'fillColor': 'YlGnBu'})
    folium.Popup(f"<i>Name: {r['name']}, Rating: {r['rating']}</i>").add_to(geo_j)
    folium.Tooltip(f"<i>Cafe Name: {r['name']}</i>").add_to(geo_j)

    geo_j.add_to(m)
    
    
# folium.Marker([44.9779006, -93.1831530],popup="<i>Mohsen's home</i>", tooltip="<i>Mohsen's home</i>", icon=folium.Icon(color='red',icon_color='#FFFF00')).add_to(m)
folium.Marker([44.9779006, -93.1831530], tooltip="<i>Mohsen's home</i>", icon=folium.Icon(color='red',icon_color='#FFFF00')).add_to(m)
folium.Circle([44.9779006, -93.1831530], radius=3000, tooltip="<i>Mohsen's home 3000m boundary</i>").add_to(m)

m

In [5]:
# Download data from Minnesota Geospatial Commons

mplsActiveSitesURL = 'http://services.pca.state.mn.us/api/v1/wimn/sites?activeSite=Y&format=json' #REST API

r = requests.get(mplsActiveSitesURL)
mplsSites = r.json()

# pprint(mplsSites['data'])

sites = pd.DataFrame(mplsSites['data'])
sites = sites[(sites.long != 'null') & (sites.lat != 'null') & (sites.cityName == 'St. Paul')]
sitesGeo = gpd.GeoDataFrame(sites, geometry=gpd.points_from_xy(sites.long, sites.lat))
sitesGeo.head()


,zipCode,watershedUrl,huc8,lat,siteName,coordinateMethod,legislativeDistrict,watershedName,countyName,cityName,...,stateCode,institutionalControl,addressLine2,ownerName,activeSite,siteId,addressLine1,ctuName,industrialClassification,geometry
9439,55120,http://www.pca.state.mn.us/index.php/water/wat...,07020012,44.867066,BRRT Mendota Heights Trailhead,Digitized - MPCA online map,52B,Lower Minnesota River,Dakota,St. Paul,...,MN,N,null,Dakota County,Y,247507,1498 Mendota Heights Road,Mendota Heights,null,POINT (-93.17332 44.86707)
10380,55106,http://www.pca.state.mn.us/index.php/water/wat...,07010206,44.963278,Beacon Bluff Parcel 2,Digitized - MPCA online map,67A,Mississippi River - Twin Cities,Ramsey,St. Paul,...,MN,N,null,null,Y,217229,833 Minnehaha Ave E.,Saint Paul,null,POINT (-93.06525 44.96328)
13484,55111,http://www.pca.state.mn.us/index.php/water/wat...,07020012,44.88351964,Boy Scouts of America Northern Star Council Le...,Zip Code Centroid,63B,Lower Minnesota River,Hennepin,St. Paul,...,MN,N,null,Scouting USA,Y,216143,201 Bloomington Road,Fort Snelling Unorganized,null,POINT (-93.18878 44.88352)
23734,55105,http://www.pca.state.mn.us/index.php/water/wat...,07010206,44.94021,Commercial Building,Digitized - MPCA online map,64A,Mississippi River - Twin Cities,Ramsey,St. Paul,...,MN,Y,null,null,Y,233050,1037 Grand Avenue,Saint Paul,null,POINT (-93.14389 44.94021)
23888,55103,http://www.pca.state.mn.us/index.php/water/wat...,07010206,44.9847,Como Golf Course Stormwater BMPs,Digitized - MPCA online map,66B,Mississippi River - Twin Cities,Ramsey,St. Paul,...,MN,N,null,CRWD,Y,235138,"1431 Lexington Pkwy N,",Saint Paul,null,POINT (-93.15403 44.98470)


In [6]:
# making a polygon feature of the buffer of 1.5 km for each site and make map
lat_mean = (coffee_shops_gdf['lat'].astype('float').mean()+sitesGeo['lat'].astype('float').mean())/2
lng_mean = (coffee_shops_gdf['lng'].astype('float').mean()+sitesGeo['long'].astype('float').mean())/2
m.location = [lat_mean, lng_mean] # set the location to the average of all points

sitesGeo.crs = 4326
sitesGeo['buffer'] = sitesGeo.to_crs('epsg:3174').buffer(1500).to_crs('epsg:4326') #building a buffer of 1.5 km around each site

for _, r in sitesGeo.iterrows():
    sim_geo = gpd.GeoSeries(r['buffer']) 
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, 
                           style_function = lambda x: {'color': 'red', 'weight': 1,  'fillColor': 'yellow'})
    folium.Marker([r['geometry'].y, r['geometry'].x],
                  popup=f"<i>Site Name: {r['siteName']}<br>Classification: {r['industrialClassification']}</i>", 
                  tooltip=f"<i>Site Name: {r['siteName']}</i>", icon=folium.Icon(color='pink')).add_to(m)
    geo_j.add_to(m)

m


In [7]:
# Running inner spatial join to find the coffeshop within the MPCA site areas

sitesGeo = sitesGeo.set_geometry('buffer') # set the buffered area to be the main geometry column

sjoin_inner = sjoin(coffee_shops_gdf, sitesGeo, how="inner")
sjoin_inner.head()


,name,business_status,open_now,rating,address,lat_left,lng,geometry_left,index_right,zipCode,...,stateCode,institutionalControl,addressLine2,ownerName,activeSite,siteId,addressLine1,ctuName,industrialClassification,geometry_right
0,Alle Aamin Coffee Shop,OPERATIONAL,True,4.5,"609 Cedar Ave, Minneapolis",44.967374,-93.247063,POINT (-93.24706 44.96737),92256,55404,...,MN,N,null,Quality Carriers,Y,234423,I-94 W MM 234,Minneapolis,"General Freight Trucking, Long-Distance, Truck...",POINT (-93.26017 44.96155)
1,Midway Coffee and Tea,OPERATIONAL,True,5.0,"1607 University Ave W, St Paul",44.956069,-93.168279,POINT (-93.16828 44.95607),31657,55104,...,MN,N,null,Saint Paul city of,Y,217646,1753 University Ave W,Saint Paul,null,POINT (-93.17463 44.95579)
3,Starbucks,OPERATIONAL,False,3.8,"1536 Hewitt Ave, St Paul",44.964455,-93.166626,POINT (-93.16663 44.96446),31657,55104,...,MN,N,null,Saint Paul city of,Y,217646,1753 University Ave W,Saint Paul,null,POINT (-93.17463 44.95579)
11,Starbucks,OPERATIONAL,False,3.8,"234 Snelling Ave N, St Paul",44.948251,-93.166686,POINT (-93.16669 44.94825),31657,55104,...,MN,N,null,Saint Paul city of,Y,217646,1753 University Ave W,Saint Paul,null,POINT (-93.17463 44.95579)
19,Ginkgo Coffeehouse,OPERATIONAL,False,4.4,"721 Snelling Ave N, St Paul",44.963016,-93.167383,POINT (-93.16738 44.96302),31657,55104,...,MN,N,null,Saint Paul city of,Y,217646,1753 University Ave W,Saint Paul,null,POINT (-93.17463 44.95579)


In [8]:
# Plotting the inner spatial join results
# The black points are the final resulting points
for _, r in sjoin_inner.iterrows():

    folium.Marker([r['geometry_left'].y, r['geometry_left'].x],
                  popup=f"<i>Name: {r['name']}, Rating: {r['rating']}</i>", 
                  tooltip=f"<i>Cafe Name: {r['name']}</i>", icon=folium.Icon(color='black')).add_to(m)
m


In [9]:
# Download meteorological data from NDAWN
import requests
from urllib.parse import urlencode
import pandas as pd 
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import folium
def get_weather(start_date, end_date, station_count = None):
    df = []
    row = 1
    for station in range(1, station_count):
        base_url = 'https://ndawn.ndsu.nodak.edu/table.csv?'
        params = f"station={str(station)}&begin_date={'2021-09-23'}&end_date={'2021-09-23'}&ttype=daily&quick_pick=&variable=ddmxt&variable=ddmnt&variable=ddavt&variable=ddbst&variable=ddtst&variable=ddws&variable=ddmxws&variable=ddsr&variable=ddr&variable=dddp&variable=ddwc"
        url = base_url+params
        r = requests.get(url)

        with open("response.csv", "w") as text_file:
            text_file.write(r.content.decode('utf-8'))

        if row == 1:
            instance = pd.read_csv('response.csv', header=[0,1],skiprows=3)
            if instance.iloc[0][0].find('<') == -1: 
                df = instance
                row = 0
        else:
            instance = pd.read_csv('response.csv', header=[0,1],skiprows=3)
            df = pd.concat([df, instance], ignore_index=True)
    
    return df
            
start_date = '2021-09-23'
end_date = '2021-09-23'
df = get_weather(start_date, end_date, 30) # get the data for 30 ND weather stations
df.columns = [f'{i}_{j}' for i, j in df.columns] # concat two header rows into one 

df.to_csv('weather.csv', encoding='utf-8', index=False)

stations_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude_deg, df.Latitude_deg))
stations_gdf.crs = 4326

stations_gdf.head()



,Station Name_Unnamed: 0_level_1,Latitude_deg,Longitude_deg,Elevation_ft,Year_Unnamed: 4_level_1,Month_Unnamed: 5_level_1,Day_Unnamed: 6_level_1,Max Temp_Degrees F,Max Temp Flag_Unnamed: 8_level_1,Min Temp_Degrees F,...,Max Wind Speed Flag_Unnamed: 20_level_1,Total Solar Rad_Lys,Total Solar Rad Flag_Unnamed: 22_level_1,Rainfall_inch,Rainfall Flag_Unnamed: 24_level_1,Dew Point_Degrees F,Dew Point Flag_Unnamed: 26_level_1,Wind Chill_Degrees F,Wind Chill Flag_Unnamed: 28_level_1,geometry
0,Eldred,47.68800,-96.822000,861,2021,9,23,77.324,NaN,49.874,...,NaN,415.359,NaN,0.0,NaN,44.711,NaN,61.818,NaN,POINT (-96.82200 47.68800)
1,Perley,47.17900,-96.680000,895,2021,9,23,76.460,NaN,48.776,...,NaN,401.288,NaN,0.0,NaN,44.803,NaN,62.161,NaN,POINT (-96.68000 47.17900)
2,Humboldt,48.88400,-97.150000,798,2021,9,23,79.376,NaN,48.578,...,NaN,395.297,NaN,0.0,NaN,43.699,NaN,60.867,NaN,POINT (-97.15000 48.88400)
3,Stephen,48.45675,-96.853953,1072,2021,9,23,77.684,NaN,51.422,...,NaN,412.969,NaN,0.0,NaN,43.965,NaN,62.420,NaN,POINT (-96.85395 48.45675)
4,Warren,48.13700,-96.839000,854,2021,9,23,76.802,NaN,49.532,...,NaN,413.463,NaN,0.0,NaN,44.079,NaN,60.449,NaN,POINT (-96.83900 48.13700)


In [10]:
# plot an interactive map visualizing all downloaded data on the same map.

for _, r in stations_gdf.iterrows():

    sim_geo = gpd.GeoSeries(r['geometry']) 
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, 
                           style_function = lambda x: {'color': 'red', 'weight': 1,  'fillColor': 'YlGnBu'})
    folium.Popup(f"<i>Station: {r['Station Name_Unnamed: 0_level_1']}, Max Temperature: {r['Max Temp_Degrees F']}</i>").add_to(geo_j)
    folium.Tooltip(f"<i>Station: {r['Station Name_Unnamed: 0_level_1']}</i>").add_to(geo_j)
    geo_j.add_to(m)


sw = [44.867066, -104.248]
ne = [48.884, -93.065245]

m.fit_bounds([sw, ne]) # adjust the zoom level to the map extent to include all points added to the map from the beginning. 

m
